In [222]:
import pandas as pd
import numpy as np
import recordlinkage
from datetime import datetime

In [223]:
df_residencia = pd.read_excel('residencia_final.xlsx')
df_procedencia = pd.read_excel('procedencia_final.xlsx')

In [224]:
del df_residencia['Unnamed: 0']
del df_residencia['Unnamed: 0.1']
del df_procedencia['Unnamed: 0']
del df_procedencia['Unnamed: 0.1']

In [225]:
df_residencia.head()

,LLAVE,TIPO_NIVEL,FECHA_NACIMIENTO,SEXO,ESTADO_CIVIL,DEPARTAMENTO_RESIDENCIA,DIRECCION_RESIDENCIA,TEL_RESIDENCIA
0,314246,POSGRADO,1987-05-21,F,SOLTERO,NaN,CL. 80 # 62 - 54,3043891301
1,293884,POSGRADO,1982-11-17,F,SOLTERO,BOGOTA D.C.,CL. 126 # 52 - 92,6240608
2,344641,POSGRADO,1989-08-08,M,SOLTERO,BOGOTA D.C.,CL. 52 # 24 - 35,7675289
3,339861,PREGRADO,1988-02-22,M,NAN,CUNDINAMARCA,MANZANA F CASA N 15 URBANIZACION LA ALEJANDRIA,3203761470
4,332394,POSGRADO,1985-10-16,F,SOLTERO,BOGOTA D.C.,TRAV. 86 C # 62 - 21,5738128


In [226]:

df_procedencia.head()

,LLAVE,TIPO_NIVEL,EDAD,ESTADO_CIVIL,TELEFONO_MOVIL,TELEFONO,DEPTO_PROCEDENCIA,DIRECCION_PROCEDENCIA,TEL_PROCEDENCIA,EXTRANJERO
0,314246,POSGRADO,34.000000,SOLTERO,3043891301,3037591,BOGOTA D.C.,CL. 80 # 62 - 54,2404337,NO
1,293884,POSGRADO,39.000000,SOLTERO,NaN,NaN,BOGOTA D.C.,CL. 125 # 40 - 82,6240608,NO
2,344641,POSGRADO,32.000000,SOLTERO,NaN,NaN,BOGOTA D.C.,CL. 52 # 24 - 35,7697210,NO
3,339861,PREGRADO,33.000000,SOLTERO,3203761470,NaN,BOYACA,CRA. 10 # 12 - 53,3102090338,NO
4,332394,POSGRADO,26.220676,SOLTERO,NaN,NaN,BOGOTA D.C.,TRAV. 86 C # 62 - 21,5738128,NO


In [227]:
df_procedencia['EDAD'] = df_procedencia['EDAD'].astype(int)
df_procedencia['ESTADO_CIVIL'] = list(map(lambda x: np.nan if x=='NAN' else x, df_procedencia['ESTADO_CIVIL']))
df_procedencia['TELEFONO_MOVIL'] = list(map(lambda x: np.nan if x=='nan' else x, df_procedencia['TELEFONO_MOVIL']))
df_procedencia['TELEFONO'] = list(map(lambda x: np.nan if x=='nan' else x, df_procedencia['TELEFONO']))
df_procedencia['DEPTO_PROCEDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_procedencia['DEPTO_PROCEDENCIA']))
df_procedencia['TEL_PROCEDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_procedencia['TEL_PROCEDENCIA']))
df_procedencia['DIRECCION_PROCEDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_procedencia['DIRECCION_PROCEDENCIA']))

In [228]:
df_residencia['ESTADO_CIVIL'] = list(map(lambda x: np.nan if x=='NAN' else x, df_residencia['ESTADO_CIVIL']))
df_residencia['DEPARTAMENTO_RESIDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_residencia['DEPARTAMENTO_RESIDENCIA']))
df_residencia['TEL_RESIDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_residencia['TEL_RESIDENCIA']))
df_residencia['DIRECCION_RESIDENCIA'] = list(map(lambda x: np.nan if x=='nan' else x, df_residencia['DIRECCION_RESIDENCIA']))


In [229]:
df_procedencia['TELEFONO_INDEX'] = df_procedencia.apply(lambda x: x[4] if str(x[4]) != 'nan' else x[8], axis=1)

In [230]:
df_procedencia['INDICE'] = df_procedencia.apply(lambda x: str(x[1]) + str(x[2]) + str(x[3]) + str(x[10]), axis=1)

In [231]:
hoy = np.datetime64(datetime.now().date())
diferencia_fechas = hoy - df_residencia['FECHA_NACIMIENTO']
diferencia_fechas = diferencia_fechas.astype('timedelta64[Y]')

In [232]:
df_residencia['EDAD_INDEX'] = diferencia_fechas.astype(int)

In [233]:
df_residencia['INDICE'] = df_residencia.apply(lambda x: str(x[1]) + str(x[8]) + str(x[4]) + str(x[7]), axis=1)

In [234]:
indexer = recordlinkage.Index()

In [235]:
indexer.sortedneighbourhood(left_on='INDICE', right_on='INDICE', window=11)

<Index>

In [236]:
candidatos = indexer.index(df_residencia, df_procedencia)

In [239]:
df_residencia[df_residencia.index == 3]

,LLAVE,TIPO_NIVEL,FECHA_NACIMIENTO,SEXO,ESTADO_CIVIL,DEPARTAMENTO_RESIDENCIA,DIRECCION_RESIDENCIA,TEL_RESIDENCIA,EDAD_INDEX,INDICE
3,339861,PREGRADO,1988-02-22,M,NaN,CUNDINAMARCA,MANZANA F CASA N 15 URBANIZACION LA ALEJANDRIA,3203761470,33,PREGRADO33nan3203761470


In [75]:
df_procedencia[df_procedencia.index == 24879]

,LLAVE,TIPO_NIVEL,EDAD,ESTADO_CIVIL,TELEFONO_MOVIL,TELEFONO,DEPTO_PROCEDENCIA,DIRECCION_PROCEDENCIA,TEL_PROCEDENCIA,EXTRANJERO,TELEFONO_INDEX,INDICE
24879,474282,PREGRADO,33,NaN,3226577286,5681308,ANTIOQUIA,CRA. 9 # 7 - 74,5563597,NO,3226577286,PREGRADO33nan3226577286


In [240]:
compare = recordlinkage.Compare()

In [283]:
compare = recordlinkage.Compare()
compare.exact('TIPO_NIVEL', 'TIPO_NIVEL', label='TIPO_NIVEL')
compare.exact('EDAD_INDEX',
               'EDAD',
               label='EDAD')
compare.exact('ESTADO_CIVIL',
               'ESTADO_CIVIL',
               label='ESTADO_CIVIL')
compare.string('TEL_RESIDENCIA',
                'TELEFONO_INDEX',
               method='jarowinkler',
               threshold=0.85,
               label='TELEFONO')
compare.string('INDICE',
                'INDICE',
               method='jarowinkler',
               threshold=0.95,
               label='patron')
features = compare.compute(candidatos, df_residencia, df_procedencia)


In [315]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

5.0     16610
4.0    259807
3.0    349925
2.0     28257
1.0      6907
0.0         3
dtype: int64

In [316]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()

In [317]:
potential_matches

,level_0,level_1,TIPO_NIVEL,EDAD,ESTADO_CIVIL,TELEFONO,patron
0,0,20325,1,1,1,0.0,0.0
1,3,44682,1,1,0,0.0,0.0
2,5,36069,1,1,1,0.0,0.0
3,6,16169,1,1,1,0.0,0.0
4,9,1750,1,1,0,0.0,0.0
...,...,...,...,...,...,...,...
661501,45426,14876,1,1,1,0.0,0.0
661502,45428,33075,1,1,1,0.0,0.0
661503,45429,22128,1,1,1,0.0,0.0
661504,45430,44273,1,1,1,0.0,0.0


In [318]:
potential_matches['Score'] = potential_matches.loc[:, 'TIPO_NIVEL':'patron'].sum(axis=1)
potential_matches.head()

,level_0,level_1,TIPO_NIVEL,EDAD,ESTADO_CIVIL,TELEFONO,patron,Score
0,0,20325,1,1,1,0.0,0.0,3.0
1,3,44682,1,1,0,0.0,0.0,2.0
2,5,36069,1,1,1,0.0,0.0,3.0
3,6,16169,1,1,1,0.0,0.0,3.0
4,9,1750,1,1,0,0.0,0.0,2.0


In [319]:
df_residencia_ = df_residencia[['INDICE']].reset_index()
df_procedencia_ = df_procedencia[['INDICE']].reset_index()

In [320]:
#df_residencia_.columns= ['level_0', 'INDICE']
#df_procedencia_.columns= ['level_1', 'INDICE']

In [321]:
df_residencia_.head()

,index,INDICE
0,0,POSGRADO34SOLTERO3043891301
1,1,POSGRADO39SOLTERO6240608
2,2,POSGRADO32SOLTERO7675289
3,3,PREGRADO33nan3203761470
4,4,POSGRADO36SOLTERO5738128


In [322]:
df_procedencia_.head()

,index,INDICE
0,0,POSGRADO34SOLTERO3043891301
1,1,POSGRADO39SOLTERO6240608
2,2,POSGRADO32SOLTERO7697210
3,3,PREGRADO33SOLTERO3203761470
4,4,POSGRADO26SOLTERO5738128


In [323]:
potential_matches['id_residencia'] = potential_matches['level_0']
df_merge = potential_matches.set_index('level_0').join(df_residencia_.set_index('index'), how='left')

In [324]:
df_merge['id_procedencia'] = df_merge['level_1']
df_merge = df_merge.set_index('level_1').join(df_procedencia_.set_index('index'), how='left', lsuffix='_left')

In [325]:
df_merge = df_merge[df_merge['Score']>=5]

In [332]:
df_merge[df_merge['id_residencia'] == df_merge['id_procedencia']]

,TIPO_NIVEL,EDAD,ESTADO_CIVIL,TELEFONO,patron,Score,id_residencia,INDICE_left,id_procedencia,INDICE
0,1,1,1,1.0,1.0,5.0,0,POSGRADO34SOLTERO3043891301,0,POSGRADO34SOLTERO3043891301
1,1,1,1,1.0,1.0,5.0,1,POSGRADO39SOLTERO6240608,1,POSGRADO39SOLTERO6240608
10,1,1,1,1.0,1.0,5.0,10,POSGRADO28SOLTERO5364365,10,POSGRADO28SOLTERO5364365
16,1,1,1,1.0,1.0,5.0,16,POSGRADO28SOLTERO7486111,16,POSGRADO28SOLTERO7486111
21,1,1,1,1.0,1.0,5.0,21,PREGRADO29SOLTERO9007034,21,PREGRADO29SOLTERO9007034
...,...,...,...,...,...,...,...,...,...,...
45408,1,1,1,1.0,1.0,5.0,45408,POSGRADO31SOLTERO3177168318,45408,POSGRADO31SOLTERO3177168318
45428,1,1,1,1.0,1.0,5.0,45428,PREGRADO25SOLTERO3002940530,45428,PREGRADO25SOLTERO3002940530
45429,1,1,1,1.0,1.0,5.0,45429,PREGRADO24SOLTERO3016936698,45429,PREGRADO24SOLTERO3016936698
45446,1,1,1,1.0,1.0,5.0,45446,PREGRADO19SOLTERO3215651517,45446,PREGRADO19SOLTERO3215651517
